In [1]:
# I created this to analyze the short Volume on a daily basis

In [2]:
import pandas as pd
from pandas import DataFrame
import requests
#import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date
#import numpy as np
from openpyxl import load_workbook

In [3]:
# Am using yesterday's date to get info from Stock Volume. My part of the world, we are ahead of USA
yesterday = date.today() - timedelta(days=1) 

In [4]:
# To get data in needed format
date_t = yesterday.strftime("%Y%m%d")

In [5]:
# To get url with date 
url = 'http://regsho.finra.org/CNMSshvol'
url_v = url + date_t + '.txt'

In [6]:
# To get contents from URL and ready for Python to read
response = requests.get(f'{url_v}')
data = response.text

In [7]:
# To clean the data
data = data.replace('\r','') # Once URL contents are readable in python, must remove /r and /n from data
data = data.replace('\n','|') # /n is being replaced with | due to easier when split it up below
a = data.split('|') # All content will be split according to | for easier breakdown into columns
del a[-2:] # Last 2 items in all data are not needed

In [8]:
# Since each data has 6 columns, can break data up equally among 6 columns

# Sum of each columns(datee,symee, etc) will be total of entire data
# len(datee+symee+svee+sevee+tvee+mee) == len(a) : This needs to be true for code to work
# Below is list which will contain the iterations of 6's 
# Example >> datee will have 0,6,12,18,24,....
# symee will have 1,7,13,19,.... 
# can use this with below (Date, Symbol) to put a from above into list for columns in pandas

datee = []
symee = []
svee = []
sevee = []
tvee = []
mee = []

#This is to get all the numbers in each column to extract the data. 
# Example >> datee will have 0,6,12,18,24,....
# symee will have 1,7,13,19,....

for i in range(0,6):
    for h in range(i,len(a),6):
        if i == 0:
            datee.append(h)
        elif i == 1:
            symee.append(h)
        elif i == 2:
            svee.append(h)
        elif i == 3:
            sevee.append(h)
        elif i == 4:
            tvee.append(h)
        elif i == 5:
            mee.append(h)

In [9]:
#Just to check the quantity is correct, if not numbering and column info will be wrong 
while len(datee+symee+svee+sevee+tvee+mee) != len(a):
    print('There is an error in one of the columns')
    break
else:
    print('Sum of all columns is correct')

Sum of all columns is correct


In [10]:
# All 6 below will be columns used in pandas containing the content of the URL 
# Date will have all the dates of the URL contents, etc

Date = []
Symbol = []
Short_Volume = []
Short_Exempt_Volume = []
Total_Volume = []
Market = []

# This will get all the data and put them into their respective columns 
# Example for Date.append(a[dd]), a[dd] = a[01] = the first index of the a(data from URL) and so on

for dd in datee:
    Date.append(a[dd])
for sy in symee:
    Symbol.append(a[sy])
for sv in svee:
    Short_Volume.append(a[sv])
for sev in sevee:
    Short_Exempt_Volume.append(a[sev])
for tve in tvee:
    Total_Volume.append(a[tve])
for me in mee:
    Market.append(a[me])

In [11]:
# Creating a blank data frame and adding Data from above with names for columns
df = pd.DataFrame() 
df['Date'] = Date
df['Symbol'] = Symbol
df['Short Volume'] = Short_Volume
df['Short Exempt Volume'] = Short_Exempt_Volume
df['Total Volume'] = Total_Volume
df['Market'] = Market

df.drop([0], inplace=True) # Drop first index
df.reset_index(drop= True, inplace=True) # Resetting index numbers

#Changing the dtype above for easier data understanding (mean, std, basically, df.describe)
df = df.astype({"Short Volume":'int', "Short Exempt Volume":'int', "Total Volume":'int'}) 

df['Date'] = pd.to_datetime(df['Date'], dayfirst = True) # change date to datetime format
# To convert the date into Dtype date then only change the format below. 
# Below will change it to object 
df['Date'] = df["Date"].dt.strftime('%d %B %Y') # Changing it easier readibility 

GME = df.loc[df['Symbol'] == 'GME'] # Extracting GME numbers
AMC = df.loc[df['Symbol'] == 'AMC'] # Extracting AMC numbers

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9589 entries, 0 to 9588
Data columns (total 6 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Date                 9589 non-null   object
 1   Symbol               9589 non-null   object
 2   Short Volume         9589 non-null   int32 
 3   Short Exempt Volume  9589 non-null   int32 
 4   Total Volume         9589 non-null   int32 
 5   Market               9589 non-null   object
dtypes: int32(3), object(3)
memory usage: 337.2+ KB


In [13]:
df.describe()

,Short Volume,Short Exempt Volume,Total Volume
count,9.589000e+03,9.589000e+03,9.589000e+03
mean,2.399906e+05,4.960014e+03,5.319452e+05
std,2.039770e+06,1.119438e+05,4.047948e+06
min,1.000000e+00,0.000000e+00,1.000000e+00
25%,3.402000e+03,0.000000e+00,8.873000e+03
50%,2.065500e+04,0.000000e+00,5.148300e+04
75%,9.541500e+04,3.650000e+02,2.257590e+05
max,1.466578e+08,9.479675e+06,2.741241e+08


In [14]:
from calendar import monthrange
days = monthrange(int(yesterday.strftime("%Y")),int(yesterday.strftime("%m"))) # To find out how many days in a month

date_need = []
for i in range(1,days[1]+1):
    date_month = datetime(int(yesterday.strftime("%Y")),int(yesterday.strftime("%m")),i) # To find the first day of the month
    # Below will go thru each day in day month and if weekday, will add it to date_need
    if date_month.strftime("%A") in ['Monday','Tuesday','Wednesday','Thursday','Friday']: 
        date_need.append(date_month.strftime("%Y%m%d"))
        
count = date_need.index(yesterday.strftime("%Y%m%d")) + 1 # To find the location/count of the date in month for AMC, GME pos.

In [15]:
if date_need[0] == yesterday.strftime("%Y%m%d"): 
    # The first day of the workweek in date_need must equal to the first day of actual date
    with pd.ExcelWriter(f'Short Stocks Volume {yesterday.strftime("%B %Y")} - Finra.xlsx', engine='openpyxl') as writer:
            GME.to_excel(writer, sheet_name='GME')
            AMC.to_excel(writer, sheet_name='AMC')
            df.to_excel(writer, sheet_name=f'Short Volume {date_t}')
            writer.close()
else: 
    # Everything else will be appended
    with pd.ExcelWriter(f'Short Stocks Volume {yesterday.strftime("%B %Y")} - Finra.xlsx', mode = 'a', engine='openpyxl') as writer:  
        # try to open an existing workbook
        writer.book = load_workbook(f'Short Stocks Volume {yesterday.strftime("%B %Y")} - Finra.xlsx') 
        # copy existing sheets
        writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets) 
        # will append to existing GME sheet, sheet name here specified
        GME.to_excel(writer, 'GME',header=False,startrow=count) 
         # will append to existing AMC sheet, sheet name here specified
        AMC.to_excel(writer, 'AMC',header=False,startrow=count)
         # will create new sheet, sheet name new one given
        df.to_excel(writer, sheet_name=f'Short Volume {date_t}')
        writer.close() 